In [ ]:
# import model and everything needed to run it
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer

import os
import numpy as np
from copy import deepcopy
from rbm_torch.models.pool_crbm_base import pool_CRBM
from rbm_torch.utils.utils import load_run_file

In [ ]:
# load data from preconfigured run file
run_file = "./datasets/toy/run_files/toy_pcrbm.json"
run_data, base_config = load_run_file(run_file)

base_config["model_name"] = run_data["model_name"]
base_config["gpus"] = run_data["gpus"]

model_type = run_data["model_type"]
assert model_type == "pool_crbm"

server_model_dir = run_data["server_model_dir"]
base_config['seed']

In [ ]:
# Training Code for individual model
def train_model(config):
    model = pool_CRBM(config, debug=False, precision=config["precision"])
    logger = TensorBoardLogger(server_model_dir, name=config["model_name"])
    if config["gpus"] > 1:
        # distributed data parallel, multi-gpus on single machine or across multiple machines
        plt = Trainer(max_epochs=config['epochs'], logger=logger, gpus=run_data["gpus"], accelerator="cuda", strategy="ddp")  # distributed data-parallel
    else:
        if config['gpus'] == 0:
            plt = Trainer(max_epochs=config['epochs'], logger=logger, accelerator="cpu")
        else:
            plt = Trainer(max_epochs=config['epochs'], logger=logger, devices=run_data["gpus"], accelerator="cuda")  # gpus=1,
    plt.fit(model)
    return

In [ ]:
# define several models for comparison across a single variable
vary = "ld"
vrange = np.geomspace(0.001, 0.1, num=6).tolist()
vrange.insert(0, 0.)
vrange = [round(x, 3) for x in vrange]


model_configs = [deepcopy(base_config) for _ in vrange]

for vid, vval in enumerate(vrange):
    model_configs[vid][vary] = vval
    model_configs[vid]["model_name"] += f"_{vary}_{vid}"

In [ ]:
for vid in range(len(vrange)):
    train_model(model_configs[vid])